# This script includes the following points:

Step 1: Importing the libraries and data

Step 2. Calculating the mean of order number grouped by department.

Step 3. Comparing the results with the subset.

Step 4. Creating a Loyalty Flag for the Customers.

Step 5. Grouping costumers based on their spending habits.

Step 6. Flagging low- and high-spenders.

Step 7. Flagging frequent and nonfrequent customers.

Step 8. Exporting data in pkl format.

## Step 1. Importing the libraries and data

In [27]:
#Import libraries 
import pandas as pd
import numpy as np
import os

In [28]:
path = r'/Users/buketoztekin/Documents/Instacart Basket Analysis/'
file_path = os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl')
ords_prods_merge = pd.read_pickle(file_path)

## Step 2. Calculating the mean of order number grouped by department 

In [29]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})


,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Step 3. Comparing the result with the subset

Means of order_number are different than the subset. Some means are higher and some are lower.

## Step 4. Creating a Loyalty Flag for the Customers

In [30]:
#Creating a max_order column grouped by user_id and order_number.

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/3_/dyc73j694fb41h6xln2l5_rc0000gn/T/ipykernel_25260/1681540670.py:1: FutureWarning: The provided callable <function max at 0x1124ef9c0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [31]:
#Creating a loyalty flag based on the number of orders.
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [32]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'


In [33]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'


In [34]:
# Check frequency of values in new loyalty flag column
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

The number of the regular customers are the highest and the number of the new customers are the lowest. 

In [35]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer


## Step 5. Grouping costumers based on their spending habits

In [36]:
#Group costumers based on their spending habits.

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'std', 'median']})

prices                                 
                       mean  min      max         std median
loyalty_flag                                                
Loyal customer    10.386336  1.0  99999.0  328.017787    7.4
New customer      13.294670  1.0  99999.0  597.560299    7.4
Regular customer  12.495717  1.0  99999.0  539.720919    7.4

It appears that the spending of buyers ranges from 1 to 99999. The means for all customer groups are significantly higher than the medians, indicating the presence of a few very high spenders that cause the data to be right-skewed. The high standard deviations suggest a large variation within the groups.

The table above shows that the mean spending of new customers is the highest, followed by regular customers, and then loyal customers. The standard deviations follow the same order.

Therefore, it is advisable to conduct a separate analysis by removing the highest-spending outliers.

## Extra step. Dropping the max spending of 99999.0

In [37]:
ords_prods_merge_outliers_dropped = ords_prods_merge[ords_prods_merge['prices'] != 99999.0]

In [38]:
ords_prods_merge_outliers_dropped.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,add_to_cart_order,reordered,_merge,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,both,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,both,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,both,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,both,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,both,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer


In [39]:
#Checking the new stats on the 'prices' column.
prices_stats = ords_prods_merge_outliers_dropped['prices'].describe()
print(prices_stats)

count    3.240416e+07
mean     9.826461e+00
std      1.741449e+02
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      1.490000e+04
Name: prices, dtype: float64


In [40]:
ords_prods_merge_outliers_dropped.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'std', 'median']})

prices                                 
                       mean  min      max         std median
loyalty_flag                                                
Loyal customer     9.579349  1.0  14900.0  164.031823    7.4
New customer      10.059917  1.0  14900.0  183.440124    7.4
Regular customer   9.894716  1.0  14900.0  176.732934    7.4

The new table shows lower maximum spendings, along with lower standard deviations and means. The means and medians are much closer to each other, but the data still appears to be right-skewed. Similar to the previous table, the mean spending of new customers is the highest, followed by regular customers, and then loyal customers.

## Step 6. Flagging low- and high-spenders

In [41]:
#Create a new column for the spending habits of the customers
ords_prods_merge['average_spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [43]:
#Assign a spender tag for each customer
ords_prods_merge.loc[ords_prods_merge['average_spending'] >= 10, 'spender_flag'] = 'High spender'

In [44]:
ords_prods_merge.loc[ords_prods_merge['average_spending'] < 10, 'spender_flag'] = 'Low spender'

In [45]:
# Check spender flags
ords_prods_merge['spender_flag'].value_counts(dropna = False)

spender_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [55]:
#Checking the new column
ords_prods_merge.head(20)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spender_flag,purchase_frequency,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,7.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,Mid-range product,Regularly busy,Busiest days,Average orders,26,Regular customer,6.935398,Low spender,11.0,Regular customer
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,Mid-range product,Regularly busy,Regularly busy,Most orders,9,New customer,5.957576,Low spender,19.0,Regular customer
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,Mid-range product,Regularly busy,Slowest days,Most orders,12,Regular customer,6.680000,Low spender,6.0,Frequent customer
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,Mid-range product,Regularly busy,Busiest days,Average orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,Mid-range product,Regularly busy,Busiest days,Most orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer


## Step 7. Flagging frequent and nonfrequent customers

In [46]:
#Create a new column for the frequency of the customers
ords_prods_merge['purchase_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [47]:
#Checking the new column
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,merge_indicator,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spender_flag,purchase_frequency
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,7.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0


In [48]:
#Assign a frequency tag for each customer
ords_prods_merge.loc[ords_prods_merge['purchase_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [50]:
ords_prods_merge.loc[(ords_prods_merge['purchase_frequency'] > 10) & (ords_prods_merge['purchase_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [51]:
ords_prods_merge.loc[ords_prods_merge['purchase_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [52]:
# Check frequency flags
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        22790523
Regular customer          6927608
Non-frequent customer     2686728
Name: count, dtype: int64

In [56]:
#Checking the new column
ords_prods_merge.head(20)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spender_flag,purchase_frequency,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,7.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,Mid-range product,Regularly busy,Busiest days,Average orders,26,Regular customer,6.935398,Low spender,11.0,Regular customer
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,Mid-range product,Regularly busy,Regularly busy,Most orders,9,New customer,5.957576,Low spender,19.0,Regular customer
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,...,Mid-range product,Regularly busy,Slowest days,Most orders,12,Regular customer,6.680000,Low spender,6.0,Frequent customer
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,...,Mid-range product,Regularly busy,Busiest days,Average orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,...,Mid-range product,Regularly busy,Busiest days,Most orders,20,Regular customer,7.162500,Low spender,9.5,Frequent customer


## Step 8. Exporting the new dataframe

In [57]:
# Check shape before exporting
ords_prods_merge.shape

(32404859, 25)

In [58]:
# Export the updated dataframe as a pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined_merged_derived.pkl'))